### Import dependencies


In [ ]:
# File management
import os
import glob
import requests
import json

# Audio processing
import librosa as li
from scipy import signal
import soundfile as sf

# Plotting
from matplotlib import pyplot as plt

# Machine Learning
import tensorflow as tf
import tensorflow_io as tfio

# Other
import time
import random
import numpy as np

User defined constants.


In [ ]:
# Target sample rate for resampling audio files
SAMPLE_RATE = 16000

# Target length for audio segments (in seconds)
SAMPLE_LENGTH = 3

### Download argentinian birds dataset from Xeno-Canto


#### Query Xeno-Canto API and save response as JSON file


In [ ]:
dataset_location = "..\\datasets\\xeno-canto_argentina\\"

# Query variables
country = "argentina"
group = "birds"
length = "2-60"
since = "2010-01-01"
q = "A"

In [ ]:
url = "https://xeno-canto.org/api/2/recordings?query="
params = f"cnt:{country}+grp:{group}+len:{length}+since:{since}+q:{q}"

response = requests.get(url + params)

print(f"• Query result: status-code {response.status_code}")

if response.status_code == 200:
    data = response.json()

    n_rec = data["numRecordings"]
    pages = data["numPages"]
    print(f"• Found {n_rec} recordings in {pages} pages.")

In [ ]:
# Write json files for all pages
for p in range(1, pages + 1):
    response = requests.get(url + params + f"&page={p}")
    data = response.json()

    filename = f"query_{str(p)}.json"
    with open(dataset_location + filename, "w") as file:
        json.dump(data, file, sort_keys=True, indent=4)
        print(f"• Saved page {p} as {filename}")

#### Download files to dataset audio folder


Each category will be downloaded to their corresponding subfolder.


In [ ]:
# Create audio folder inside dataset.
audio_location = dataset_location + "unprocessed_audio_files\\"
try:
    os.mkdir(audio_location)
    print(f"Created {audio_location}")
except:
    print("Folder already existed.")

In [ ]:
for file in os.listdir(dataset_location):
    if file.endswith(".json"):
        with open(dataset_location + file) as f:
            data = json.load(f)
            recordings = data["recordings"]
            print(f"Downloading files from {file}...")

            for r in recordings:
                # Get metadata from json
                id = r["id"]
                # Optional, if coloquial name is preferred
                english_name = r["en"]
                scientific_name = f"{r['gen']} {r['sp']}"
                download = r["file"]
                ext = "." + r["file-name"].split(".")[-1]

                # Create subfolder if not exists
                subfolder = scientific_name + "\\"
                try:
                    os.mkdir(audio_location + subfolder)
                except:  # noqa: E722
                    pass

                # Download file
                with open(
                    audio_location + subfolder + scientific_name + "_" + id + ext, "wb"
                ) as out_file:
                    content = requests.get(download, stream=True).content
                    out_file.write(content)

                # Wait required time between recordings (randomized)
                time.sleep(random.uniform(1.01, 1.2))

        print("Done!")

    # Wait some time between json pages (randomized)
    time.sleep(random.uniform(1, 5))

### Preprocess audio files


#### Function definitions


In [ ]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype="high", analog=False)
    return b, a


def apply_butter_highpass(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
def remove_silence(signal, thresh=18, hop=2048, plot=False):
    splits = li.effects.split(
        y=signal, top_db=thresh, frame_length=(hop * 2), hop_length=hop
    )

    # For fine-tuning purposes
    if plot:
        peak = np.max(signal)
        plt.subplots(figsize=(12, 4))
        plt.plot(signal)
        plt.vlines(splits, ymin=-peak, ymax=peak, color="red")
        plt.show()

    stripped_audio = []

    for s in splits:
        split = signal[s[0] : s[1]]
        stripped_audio.extend(split)

    return np.asarray(stripped_audio)

In [ ]:
# Target length must be in seconds
def split_audio(signal, target_length, samplerate, plot=False):

    duration = li.get_duration(y=signal, sr=samplerate)

    n_segments = np.ceil(duration / target_length)

    audio_segments = []

    for n in range(int(n_segments)):

        s = signal[
            samplerate * n * target_length : samplerate * (n + 1) * target_length
        ]

        if len(s) < target_length * samplerate:

            s = np.pad(s, (0, target_length * samplerate - len(s)), "constant")

        audio_segments.append(s)

        if plot:

            plt.plot(s, alpha=1 / n_segments)

    if plot:

        plt.show()

    return audio_segments

#### Audio preprocessing


- Load each file from all unprocessed audio subfolders.
- Make an empty copy of the folder in the processed directory.
- Apply high filter, strip noise sections and split into constant length segments.
- Save shorter segments as new WAV files.


In [ ]:
processed_location = dataset_location + "processed_audio_files\\"
subfolders = os.listdir(audio_location)

# Make processed audio folder
try:
    os.mkdir(processed_location)
except:  # noqa: E722
    pass

In [ ]:
# Optional list to filter species
wanted_species = [
    "Anas bahamensis",
    "Bubulcus ibis",
    "Caracara plancus",
    "Coereba flaveola",
    "Columba livia",
    "Columbina talpacoti",
    "Coragyps atratus",
    "Crotophaga ani",
    "Egretta thula",
    "Furnarius rufus",
    "Haematopus palliatus",
    "Jacana jacana",
    "Larus dominicanus",
    "Mimus saturninus",
    "Myiopsitta monachus",
    "Nannopterum brasilianum",
    "Paroaria coronata",
    "Pitangus sulphuratus",
    "Pyrocephalus obscurus",
    "Ramphastos toco",
    "Sicalis flaveola",
    "Turdus rufiventris",
    "Tyrannus melancholicus"
    "Vanellus chilensis",
    "Zenaida auriculata",
    "Zonotrichia capensis",
]

In [ ]:
for sub in subfolders:
    # Optional filter for keeping only desired species
    if sub not in wanted_species:
        continue

    files = os.listdir(audio_location + sub)

    # Create same subfolder in processed folder
    p_subfolder = processed_location + sub + "\\"

    try:
        os.mkdir(p_subfolder)
    except:
        pass

    for f in files:
        # Load audio file from unprocessed folder
        file_path = audio_location + sub + "\\" + f
        y, sr = li.load(file_path, sr=SAMPLE_RATE, mono=True, res_type="soxr_lq")

        # Apply high-pass filter
        y = apply_butter_highpass(data=y, cutoff=1000, fs=SAMPLE_RATE, order=6)

        # Delete silent sections
        y = remove_silence(y, plot=False)

        # Split into segments of desired length
        audio_segments = split_audio(
            y, target_length=SAMPLE_LENGTH, samplerate=SAMPLE_RATE
        )

        # Iterate through splitted audio segments and save each one as a separate .wav file
        for i, segment in enumerate(audio_segments):
            new_filename = f.split(".")[0] + "_" + str(i) + ".wav"
            sf.write(p_subfolder + new_filename, segment, SAMPLE_RATE, subtype="PCM_16")

### Tensorflow dataset generation, processing and data pipeline


#### Map functions definitions

Used for loading and processing audio data


In [ ]:
# Load audio file
def load_audio(file_path):
    # Load encoded audio file
    file = tf.io.read_file(file_path)

    # Decode wav file
    audio, sr = tf.audio.decode_wav(file, desired_channels=1)

    # Remove channels dimension
    audio = tf.squeeze(audio, axis=-1)
    sr = tf.cast(sr, dtype=tf.int64)

    return audio

In [ ]:
# Convert audio signal to spectrogram
def process_audio(file_path, label):
    # Load audio
    audio = load_audio(file_path)

    # Compute spectrogram
    spectrogram = tf.signal.stft(audio, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)

    return spectrogram, label

In [ ]:
# Encode string labels as int. Returns list of encoded labels and labels dictionary.
def encode_labels(labels_list):
    labels_dict = {}
    encoded_labels = []

    for i, label in enumerate(set(labels_list)):
        labels_dict[label] = i

    for label in labels_list:
        encoded_labels.append(labels_dict[label])

    return encoded_labels, labels_dict

#### Create TF Data Dataset


In [ ]:
processed_location = "..\\datasets\\xeno-canto_argentina\\processed_audio_files\\"

In [ ]:
# Encode labels as ints for using in ML model
regex = "\\*\\*.wav"
audio_filepaths = glob.glob(processed_location + regex)

str_labels = list(map(lambda x: x.split("\\")[-2], audio_filepaths))
encoded_labels, labels_dict = encode_labels(str_labels)

In [ ]:
audio_ds = tf.data.Dataset.list_files(processed_location + "*\\*.wav", shuffle=False)



labels = tf.data.Dataset.from_tensor_slices(encoded_labels)



# Zip together audio dataset with labels dataset



ds = tf.data.Dataset.zip((audio_ds, labels))

Test and visualize load and process audio functions


In [ ]:
filepath, label = ds.shuffle(1000).as_numpy_iterator().next()
spectrogram, label = process_audio(filepath, label)

plt.figure(figsize=(10, 6))
plt.imshow(tf.transpose(spectrogram)[0])
plt.show()

#### Build data pipeline and split into train/test subsets


In [ ]:
# Process and batch dataset
ds = ds.map(process_audio)
ds = ds.cache()
ds = ds.shuffle(buffer_size=10000)
ds = ds.batch(16)
ds = ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Split dataset
SPLIT = 0.7

train_split = int(len(ds) * SPLIT)
train_ds = ds.take(train_split)
test_ds = ds.skip(train_split)

### Create and train CNN model


In [ ]:
unique_class_count = len(labels_dict)

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(1491, 257, 1)),
        tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=128, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=unique_class_count, activation="softmax"),
    ]
)

model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
model.summary()

In [ ]:
EPOCHS = 30

# Training checkpoints
checkpoint_filepath = "../models/checkpoints/"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    verbose=0,
)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds,
    callbacks=[model_checkpoint_callback],
)

#### Plotting model accuracy and loss


In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.grid()
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.grid()
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")

figure = plt.gcf()
plt.show()

#### Save and load Keras models


##### Save model


In [ ]:
model_name = str(input("Enter a title for the saved model..."))
model_path = f"../models/{model_name}/"

if model_name:
    try:
        os.mkdir(model_path)
        model.save(model_path + f"{model_name}.keras")
        print(f"Model successfully saved to {model_path}{model_name}.")

        # Remember to execute plotting cell first
        figure.savefig(model_path + f"{model_name}.png")
    except:
        raise (
            "ERROR: Model name is already in use. Pick another name and try again."
        )

##### Load model


In [ ]:
model_name = str(input("Type model name..."))

if model_name:
    model_path = f"../models/{model_name}/"

# model = tf.keras.models.load_model(model_path + f"{model_name}.keras")

### Model testing and prediction


In [ ]:
# Load mp3 stereo file for making predictions
def load_mp3_for_prediction(filename):
    # Load file
    iot = tfio.audio.AudioIOTensor(filename)

    # Convert to tensor and combine channels to mono
    tensor = iot.to_tensor()
    tensor = tf.math.reduce_sum(tensor, axis=1) / 2

    # Get sample rate
    sample_rate = iot.rate
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)

    # Resample to target sample rate
    wav = tfio.audio.resample(tensor,
                              rate_in=sample_rate,
                              rate_out=SAMPLE_RATE)
    return wav


# Convert slices of audio into spectrograms


def process_slices(sample, index):
    sample = sample[0]
    zero_padding = tf.zeros([48000] - tf.shape(sample), dtype=tf.float32)
    wav = tf.concat([zero_padding, sample], 0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [ ]:
prediction_audio_path = "..\\datasets\\xeno-canto_argentina\\Caracara plancus.mp3"

In [ ]:
# Load audio file for prediction
audio = load_mp3_for_prediction(prediction_audio_path)

# Get windows of audio
audio_slices = tf.keras.utils.timeseries_dataset_from_array(
    audio,
    audio,
    sequence_length=SAMPLE_RATE * SAMPLE_LENGTH,
    sequence_stride=SAMPLE_RATE * SAMPLE_LENGTH,
    batch_size=1,
)

# Convert audio slices to spectrograms
samples, index = audio_slices.as_numpy_iterator().next()
audio_slices = audio_slices.map(process_slices)
audio_slices = audio_slices.batch(32)

# Make predictions for each audio slice
predictions = model.predict(audio_slices)

In [ ]:
for p in predictions:
    label = p.argmax()
    label_str = list(labels_dict.keys())[label]
    probability = np.round(p.max() * 100)

    print(label, probability)